In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
import numpy as np
import time
import csv
import os
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score

# --- GCN Model Tanımı ---
class GCN_Community(torch.nn.Module):
    def __init__(self, num_features, hidden_dim=64, embedding_dim=32):
        super(GCN_Community, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, embedding_dim)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # 1. Katman
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # 2. Katman
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # 3. Katman (Embedding Çıktısı)
        x = self.conv3(x, edge_index)
        return x

# --- GAT Model Tanımı ---
class GAT_Community(torch.nn.Module):
    def __init__(self, num_features, hidden_dim=64, embedding_dim=32, heads=4):
        super(GAT_Community, self).__init__()
        self.conv1 = GATConv(num_features, hidden_dim, heads=heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * heads, embedding_dim, heads=1, concat=False, dropout=0.6)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# --- Negatif Örnekleme (Loss fonksiyonu için) ---
def negative_sampling(data, num_neg_samples=None):
    num_nodes = data.num_nodes
    num_edges = data.edge_index.size(1)
    if num_neg_samples is None:
        num_neg_samples = num_edges
    
    neg_edges = []
    edge_set = set(map(tuple, data.edge_index.t().tolist()))
    
    while len(neg_edges) < num_neg_samples:
        i = np.random.randint(0, num_nodes)
        j = np.random.randint(0, num_nodes)
        if i != j and (i, j) not in edge_set and (j, i) not in edge_set:
            neg_edges.append([i, j])
    
    return torch.tensor(neg_edges, dtype=torch.long).t().to(data.edge_index.device)

# --- Eğitim Fonksiyonu ---
def train_gnn(model, data, optimizer, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        embeddings = model(data)
        
        # Reconstruction Loss
        pos_edge_index = data.edge_index
        neg_edge_index = negative_sampling(data)
        
        # Pozitif kenarların skoru (birbirine yakın olmalı)
        pos_loss = -torch.log(
            torch.sigmoid(
                (embeddings[pos_edge_index[0]] * embeddings[pos_edge_index[1]]).sum(dim=1)
            ) + 1e-15
        ).mean()
        
        # Negatif kenarların skoru (birbirine uzak olmalı)
        neg_loss = -torch.log(
            1 - torch.sigmoid(
                (embeddings[neg_edge_index[0]] * embeddings[neg_edge_index[1]]).sum(dim=1)
            ) + 1e-15
        ).mean()
        
        loss = pos_loss + neg_loss
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
    return model

# --- Clustering ---
def cluster_embeddings(embeddings, num_clusters=None, max_clusters=20):
    # Tensor kontrolü
    if torch.is_tensor(embeddings):
        embeddings_np = embeddings.detach().cpu().numpy()
    else:
        embeddings_np = embeddings
    min_clusters=3
    best_score = -1
    best_k = min_clusters # Varsayılan başlangıç
    
    print(f"En iyi küme sayısı aranıyor ({min_clusters}-{max_clusters} arası)...")
    
    # Aramaya min_clusters'dan başla
    search_range = range(min_clusters, min(max_clusters + 1, len(embeddings_np)))
    
    for k in search_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(embeddings_np)
        
        # Calinski-Harabasz kullanıyoruz (Daha yüksek k sayılarını sever)
        score = calinski_harabasz_score(embeddings_np, labels)
        
        # İstersen skorları yazdırıp görebilirsin
        # print(f"k={k}, Score={score:.4f}")
        
        if score > best_score:
            best_score = score
            best_k = k
            
    print(f"Seçilen küme sayısı: {best_k} (Score: {best_score:.4f})")
    
    # Final kümeleme
    kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(embeddings_np)
    
    communities = {}
    for label in np.unique(labels):
        communities[int(label)] = np.where(labels == label)[0].tolist()
    
    return communities

# --- YENİ: Embeddingleri Kaydetme Fonksiyonu ---
def save_embeddings_to_csv(embeddings, node_list, algorithm_name):
    """
    Embeddingleri CSV dosyasına kaydeder.
    Format: NodeID, Emb_0, Emb_1, ..., Emb_N
    """
    output_dir = "community_results"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    filename = os.path.join(output_dir, f"{algorithm_name.lower()}_embeddings.csv")
    
    # Tensor kontrolü
    if torch.is_tensor(embeddings):
        embeddings_np = embeddings.detach().cpu().numpy()
    else:
        embeddings_np = embeddings_np

    try:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            
            # Header yaz (NodeID, dim0, dim1...)
            header = ["NodeID"] + [f"dim_{i}" for i in range(embeddings_np.shape[1])]
            writer.writerow(header)
            
            # Her satırı yaz
            for i, node_id in enumerate(node_list):
                row = [node_id] + list(embeddings_np[i])
                writer.writerow(row)
                
        print(f"Embeddingler kaydedildi: {filename}")
        return True
    except Exception as e:
        print(f"Embedding kaydetme hatası: {str(e)}")
        return False

def save_communities_to_csv(communities, node_list, algorithm_name):
    if not communities: return False
    
    output_dir = "community_results"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    filename = os.path.join(output_dir, f"{algorithm_name.lower()}_communities.csv")
    
    try:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for comm_id, member_indices in communities.items():
                # İndeksleri gerçek düğüm isimlerine çevir
                real_members = [node_list[idx] for idx in member_indices]
                writer.writerow(real_members)
        return True
    except Exception as e:
        print(f"Community CSV hatası: {str(e)}")
        return False

def run_gnn_community_detection(G_nx, model_type="gcn", epochs=200, embedding_dim=32):
    print(f"\n{model_type.upper()} modeli çalıştırılıyor...")
    start_time = time.time()
    
    # Düğüm sırasını sabitle (NetworkX node order)
    # Bu liste, index (0,1,2) ile gerçek düğüm ismi (UserA, UserB) arasındaki eşleşmedir.
    node_list = list(G_nx.nodes())
    
    data = from_networkx(G_nx)
    
    # Node feature yoksa derece (degree) bilgisini feature olarak kullan
    if not hasattr(data, 'x') or data.x is None:
        degrees = torch.tensor([G_nx.degree(node) for node in G_nx.nodes()], dtype=torch.float)
        data.x = degrees.unsqueeze(1)
    
    num_features = data.x.size(1)
    
    if model_type.lower() == "gcn":
        model = GCN_Community(num_features, hidden_dim=64, embedding_dim=embedding_dim)
    elif model_type.lower() == "gat":
        model = GAT_Community(num_features, hidden_dim=64, embedding_dim=embedding_dim)
    else:
        print(f"Bilinmeyen model: {model_type}")
        return None, None, None
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model = train_gnn(model, data, optimizer, epochs=epochs)
    
    # Embeddingleri al
    model.eval()
    with torch.no_grad():
        embeddings = model(data)
    
    # Kümele
    communities = cluster_embeddings(embeddings)
    
    elapsed_time = time.time() - start_time
    
    # ÖNEMLİ: Embeddingleri de döndürüyoruz
    return communities, embeddings, elapsed_time, node_list

def main():
    print("Graf yükleniyor...")
    try:
        # Dosya adını kontrol et
        if not os.path.exists("my.adjlist"):
            print("HATA: 'my.adjlist' dosyası bulunamadı!")
            return

        G_nx = nx.read_adjlist("my.adjlist")
        if nx.is_directed(G_nx):
            G_nx = G_nx.to_undirected()
            
        print(f"Düğüm sayısı: {G_nx.number_of_nodes()}")
        
        gnn_models = ["gcn", "gat"]
        
        for model_type in gnn_models:
            # Fonksiyon artık 4 değer döndürüyor
            communities, embeddings, elapsed_time, node_list = run_gnn_community_detection(
                G_nx, 
                model_type=model_type,
                epochs=200,
                embedding_dim=32
            )
            
            if communities:
                algo_name = f"{model_type.upper()}_GNN"
                
                print(f"\n{algo_name} Tamamlandı ({elapsed_time:.2f} sn)")
                print(f"Toplam Küme Sayısı: {len(communities)}")
                
                # 1. Toplulukları Kaydet
                save_communities_to_csv(communities, node_list, algo_name)
                
                # 2. Embeddingleri Kaydet (YENİ)
                save_embeddings_to_csv(embeddings, node_list, algo_name)
                
        print("\nİşlem tamamlandı.")
        
    except Exception as e:
        print(f"Beklenmedik bir hata: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Graf yükleniyor...
Düğüm sayısı: 8371

GCN modeli çalıştırılıyor...
Epoch 50/200, Loss: 1.1161
Epoch 100/200, Loss: 1.3572
Epoch 150/200, Loss: 1.1465
Epoch 200/200, Loss: 1.1186
En iyi küme sayısı aranıyor (3-20 arası)...
Seçilen küme sayısı: 20 (Score: 155946.5191)

GCN_GNN Tamamlandı (2699.81 sn)
Toplam Küme Sayısı: 20
Embeddingler kaydedildi: community_results\gcn_gnn_embeddings.csv

GAT modeli çalıştırılıyor...
Epoch 50/200, Loss: 16.8393
Epoch 100/200, Loss: 14.3242
Epoch 150/200, Loss: 2.6010
Epoch 200/200, Loss: 1.5961
En iyi küme sayısı aranıyor (3-20 arası)...
Seçilen küme sayısı: 20 (Score: 183968.3589)

GAT_GNN Tamamlandı (3187.02 sn)
Toplam Küme Sayısı: 20
Embeddingler kaydedildi: community_results\gat_gnn_embeddings.csv

İşlem tamamlandı.
